Time Series with LSTM Model uygulamamız için modellerimizi kaydedeceğiz

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Veriyi Yükleyip EDA

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("airline-passengers.csv")

In [5]:
df.head()

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Month       144 non-null    object
 1   Passengers  144 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ KB


In [7]:
df.isnull().sum()

Month         0
Passengers    0
dtype: int64

In [8]:
df['Month'] = pd.to_datetime(df['Month'])

In [9]:
df.set_index('Month', inplace=True)

In [10]:
# Veri Ön İşleme

In [11]:
data = df.values.astype('float32')

In [12]:
# Normalizasyon

In [13]:
from sklearn.preprocessing import MinMaxScaler

In [14]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [15]:
scaled_data = scaler.fit_transform(data)

In [16]:
# Train/Test Ayrımı (%67 Train, %33 Test)

In [17]:
train_size = int(len(scaled_data) * 0.67)

In [18]:
test_size = len(scaled_data) - train_size

In [19]:
train_data, test_data = scaled_data[0:train_size,:], scaled_data[train_size:len(scaled_data),:]

In [20]:
# Veri Seti Oluşturma Fonksiyonu (Lookback)

In [21]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [22]:
# Geçmiş 3 aya bakarak 1 ay sonrasını tahmin et

In [23]:
look_back = 3

In [24]:
import numpy as np

In [25]:
X_train, y_train = create_dataset(train_data, look_back)

In [26]:
X_test, y_test = create_dataset(test_data, look_back)

In [27]:
# LSTM için girdiyi şekillendiriyoruz [Samples, Time Steps, Features]

In [28]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))

In [29]:
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [30]:
# LSTM Modeli

In [31]:
from tensorflow.keras.models import Sequential

In [32]:
model = Sequential()

In [33]:
from tensorflow.keras.layers import LSTM, Dense

In [34]:
model.add(LSTM(4, input_shape=(1, look_back)))

In [35]:
model.add(Dense(1))

In [36]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [37]:
# Eğitim

In [38]:
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
92/92 - 1s - 12ms/step - loss: 0.0158
Epoch 2/100
92/92 - 0s - 1ms/step - loss: 0.0073
Epoch 3/100
92/92 - 0s - 1ms/step - loss: 0.0065
Epoch 4/100
92/92 - 0s - 1ms/step - loss: 0.0057
Epoch 5/100
92/92 - 0s - 1ms/step - loss: 0.0052
Epoch 6/100
92/92 - 0s - 1ms/step - loss: 0.0048
Epoch 7/100
92/92 - 0s - 1ms/step - loss: 0.0046
Epoch 8/100
92/92 - 0s - 1ms/step - loss: 0.0043
Epoch 9/100
92/92 - 0s - 1ms/step - loss: 0.0042
Epoch 10/100
92/92 - 0s - 1ms/step - loss: 0.0041
Epoch 11/100
92/92 - 0s - 1ms/step - loss: 0.0040
Epoch 12/100
92/92 - 0s - 1ms/step - loss: 0.0040
Epoch 13/100
92/92 - 0s - 1ms/step - loss: 0.0039
Epoch 14/100
92/92 - 0s - 1ms/step - loss: 0.0038
Epoch 15/100
92/92 - 0s - 1ms/step - loss: 0.0039
Epoch 16/100
92/92 - 0s - 2ms/step - loss: 0.0038
Epoch 17/100
92/92 - 0s - 1ms/step - loss: 0.0038
Epoch 18/100
92/92 - 0s - 1ms/step - loss: 0.0038
Epoch 19/100
92/92 - 0s - 1ms/step - loss: 0.0037
Epoch 20/100
92/92 - 0s - 1ms/step - loss: 0.0037
Epoch 21

In [39]:
# Save

In [40]:
import warnings
warnings.filterwarnings('ignore')

In [41]:
model.save("lstm_airline_model.h5")

In [42]:
import joblib

In [43]:
joblib.dump(scaler, "airline_scaler.pkl")

['airline_scaler.pkl']